# Workshop on Domain-Specific Lanugages for Performance-Portable Weather and Climate Models

## Session 2B: Advanced Concepts II

### 1. Python Decorators

In [ ]:
import time

def timeit(func):
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        func(*args, **kwargs)
        print("Time: " + str(time.perf_counter() - start))
    return wrapper

@timeit
def say_whee():
    time.sleep(0.5)
    print("Whee!")

say_whee()

This is identical to the following

In [ ]:
def say_whee():
    time.sleep(0.5)
    print("Whee!")

print(repr(say_whee))
say_whee = timeit(say_whee)

# say_whee()

<div class="alert alert-block alert-info">
    <b> Now it's your turn: </b><br>
    <ul>
        <li style="margin-bottom: 10px">Extend the example above to aggregate call time to the functions decorated by <code>timeit</code>, then print a list at the end of the program.</li>
        <li style="margin-bottom: 10px">If you want a challenge, try applying it to a stencil. Note that when chaining decorators, the top decorator is the outer one.</li>
    </ul>
    Hints: You can still reference outer scope variables inside wrapper, and the following snippet may be useful to get you started
</div>

In [ ]:
import types

import numpy as np

import gt4py
import gt4py.storage as gt_storage
from gt4py import gtscript

backend = "numpy"
dtype = np.float64

N = 10
shape = (N, N, N)

def make_zeros(backend, shape, dtype=np.float64, default_origin=(0, 0, 0)):
    """Return a zero'd gt4py storage for the given backend and shape"""
    return gt_storage.zeros(backend=backend, shape=shape, dtype=dtype, default_origin=default_origin)

def get_repr(func) -> str:
    if isinstance(func, types.FunctionType):
        return func.__name__
    elif isinstance(func, gt4py.StencilObject):
        return func.definition_func.__name__ + "_" + func._gt_id_
    else:
        raise TypeError("Unrecognized type")

### 2. Transient diffusion

In this example we will discretize and solve a time-dependent diffusion problem:

$$
\begin{align}
\mathrm{Domain}: \qquad&\Omega = [0, 1]^2&\\
\mathrm{PDE}: \qquad&\frac{\partial u(\mathbf{x},t)}{\partial t} = \nabla \cdot \nabla u, \quad&&\mathbf{x} \in \Omega\\
\mathrm{IC}:\qquad &u(\mathbf{x}, 0) = 0, &&\mathbf{x} \in \Omega\\
\mathrm{BCs}:\qquad &u(\mathbf{x}, t) = 1 &&\mathbf{x} \in \Omega^{\mathrm{top}}\\
&\nabla u \cdot \mathbf{n} = 0, &&\mathbf{x} \in \partial \Omega \backslash \Omega^{\mathrm{top}}
\end{align}
$$

We will use a uniform finite difference grid of $N$ points, where the outer points of the array are the bounds of $\Omega$. Therefore $u[N-1, N-1, 0] = 1$ is a corner of the domain. We will employ a simple 5-point second-order acccurate stencil in space, and a simple first-order forward-Euler scheme in time.

GT4Py (at the moment) is desiged to work only with 3D arrays, so we will allocate storages that are a single point in the $k$ direction.

First, we start with importing packages, setting up the discretization, and defining a few helper functions.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import gt4py.storage as gt_storage
from gt4py import gtscript

from tools import print_generated_code

backend = "numpy"
dtype = np.float64

N = 128
shape = (N, N, 1)

DX = 1.0 / (N-1)
DT = DX * DX / (2 * 1) * 0.9 # Valid timestep for stability

def make_zeros(backend, shape, dtype=np.float64, default_origin=(0, 0, 0)):
    """Return a zero'd gt4py storage for the given backend and shape"""
    return gt_storage.zeros(backend=backend, shape=shape, dtype=dtype, default_origin=default_origin)

# Plotting
coord = np.linspace(0, 1, N)
X, Y = np.meshgrid(coord, coord)

def plot_solution(u):
    plt.contourf(X, Y, u.data[:, :, 0], 50)
    plt.colorbar()
    plt.show()
    
# GT4Py operator
@gtscript.function
def laplace(field):
    """Laplacian operator"""
    xd2 = -2 * field[0, 0, 0] + field[-1, 0, 0] + field[1, 0, 0]
    yd2 = -2 * field[0, 0, 0] + field[0, -1, 0] + field[0, 1, 0]
    return (xd2 + yd2) / (DX * DX)

a. First attempt: use GT4Py without regions

In [ ]:
@gtscript.stencil(backend=backend)
def take_timestep(u: gtscript.Field[dtype]):
    with computation(PARALLEL), interval(...):
        u += DT * laplace(u)


# Create storage (and set IC implicitly)
u = make_zeros(backend, shape)
for n in range(10):
    # Left
    u[0, :, :] = u[1, :, :]
    # Right
    u[-1, :, :] = u[-2, :, :]
    # Bottom
    u[:, 0, :] = u[:, 1, :]
    # Top
    u[:, N-1, :] = 1.0
    
    # Update approx. solution
    take_timestep(u, origin=(1, 1, 0))

plot_solution(u)

**What went well:** we are able to solve the PDE!

**What could be improved:** we are using numpy array operations outside stencil calls. This means code outside stencils, and will have greater latency (and probably reduced performance) than if they were part of the stencil. Consider the case of a GPU: API calls for memory copies will have to re-issue reads from global memory - which takes a long time.

<div class="alert alert-block alert-info">
    <b> Now it's your turn: </b><br>
    <ul>
        <li style="margin-bottom: 10px">Can you take the simple stencil above and add regions for the boundary conditions?</li>
        <li style="margin-bottom: 10px">Use print_generated_code to look at the code. Can you spot the regions? It may be easier if you change the backend to "debug"</li>
    </ul>
</div>

In [ ]:
@gtscript.stencil(backend=backend)
def take_timestep(u: gtscript.Field[dtype]):
    with computation(PARALLEL), interval(...):
        # HINT: Put boundary conditions here!
        # with parallel(region[...]):
        #     u[0, 0, 0] = u[...]

        u += DT * laplace(u)

# Create storage (and set IC implicitly)
u = make_zeros(backend, shape)
for n in range(100):
    take_timestep(u, origin=(1, 1, 0))

plot_solution(u)

In [ ]:
print_generated_code(take_timestep)

### 3. Bi-Laplacian operator on a cubed sphere grid

Applying the Laplace operator once on a cubed sphere accesses one ring of halo points, but no special cases need to occur in the finite difference stencil to account for corner effects.

However, on the second application of a Laplace operator (for the 4th order biharmonic operator), the operator is applied on a large domain, and indexes into the "ghost" points at the corners that do not exist. Therefore, the function that is called multiple times needs to include special conditions when including iteration over these points.

The code below sets up the operator with the corner conditions on the cubed sphere grid. The diagram below shows the operator on a corner of the grid avoiding the non-existent points. This requires a different stencil at these points, so regions need to be used!

In [ ]:
N = 128
nhalo = 2
shape = (N + 2 * nhalo, N + 2 * nhalo, 1)
origin = (2, 2, 0)
DX = 1.0 / (N-1)

@gtscript.function
def d2x(u):
    return (-2 * u[0, 0, 0] + u[-1, 0, 0] + u[1, 0, 0]) / (DX * DX)


@gtscript.function
def d2y(u):
    return (-2 * u[0, 0, 0] + u[0, -1, 0] + u[0, 1, 0]) / (DX * DX)

@gtscript.function
def lap_cube_cells(u):
    u_next = d2x(u) + d2y(u)

    # A practioner of numerical methods should rightly complain that these are committing finite difference crimes,
    # but correcting those would be overly verbose for an example.
    with parallel(region[I[0], J[0] - 1]):
        u_next = d2y(u) + (-2 * u[0, 0, 0] + u[1, 0, 0] + u[-1, 1, 0]) / (DX * DX)
    with parallel(region[I[-1], J[0] - 1]):
        u_next = d2y(u) + (-2 * u[0, 0, 0] + u[1, 0, 0] + u[1, 1, 0]) / (DX * DX)
    with parallel(region[I[0], J[-1] + 1]):
        u_next = d2y(u) + (-2 * u[0, 0, 0] + u[1, 0, 0] + u[-1, -1, 0]) / (DX * DX)
    with parallel(region[I[-1], J[-1] + 1]):
        u_next = d2y(u) + (-2 * u[0, 0, 0] + u[1, 0, 0] + u[1, -1, 0]) / (DX * DX)

    with parallel(region[I[0] - 1, J[0]]):
        u_next = d2x(u) + (-2 * u[0, 0, 0] + u[0, 1, 0] + u[1, -1, 0]) / (DX * DX)
    with parallel(region[I[-1] + 1, J[0]]):
        u_next = d2x(u) + (-2 * u[0, 0, 0] + u[0, 1, 0] + u[-1, -1, 0]) / (DX * DX)
    with parallel(region[I[0] - 1, J[-1]]):
        u_next = d2x(u) + (-2 * u[0, 0, 0] + u[0, 1, 0] + u[1, 1, 0]) / (DX * DX)
    with parallel(region[I[-1] + 1, J[-1]]):
        u_next = d2x(u) + (-2 * u[0, 0, 0] + u[0, 1, 0] + u[-1, 1, 0]) / (DX * DX)

    return u_next

If this is called only once, the regions are not activated so the code for them is not even included in the generated output. We can illustrate that by generating and looking at the code for a single call to this function:

In [ ]:
@gtscript.stencil(backend=backend)
def lap_tmp(u: gtscript.Field[dtype]):
    with computation(PARALLEL), interval(...):
        u = lap_cube_cells(u)

print_generated_code(lap_tmp)

On the other hand, if the function is called twice, the values of `u_next` resulting from the first call are used in the second call, so the regions are used in the generated code:

In [ ]:
@gtscript.stencil(backend=backend)
def biharmonic(u: gtscript.Field[dtype]):
    with computation(PARALLEL), interval(...):
        u1 = lap_cube_cells(u)
        u = lap_cube_cells(u1)

print_generated_code(biharmonic)

#### Distributed computation

What we've seen so far is that regions can be used to express computation relative to the stencil compute domain boundaries, but it can also be used to restrict computation to certain MPI ranks. AxisOffsets like `I[0]` can be compile-time externals in a stencil. If these are set to `None` and used in a region, that region is automatically excluded.

<div class="alert alert-block alert-info">
    <b> Now it's your turn: </b><br>
    Imagine you are editing <code>lap_cube_cells</code> for a 3x3 processor layout on each tile. How would it look? Familiarize yourself with the syntax for now. Tomorrow you will continue this example using fv3gfs-util to execute it on the cubed sphere.
</div>